In [ ]:
pip install numpy pandas matplotlib scikit-learn tensorflow


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# Generate synthetic data
rng = np.random.RandomState(42)
normal_data = rng.normal(loc=0.0, scale=1.0, size=(1000, 2))  # Normal data
anomaly_data = rng.uniform(low=-4, high=4, size=(50, 2))  # Anomalous data

# Combine the datasets
data = np.concatenate([normal_data, anomaly_data], axis=0)
labels = np.concatenate([np.zeros(len(normal_data)), np.ones(len(anomaly_data))])

# Visualize the dataset
plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='coolwarm', alpha=0.7)
plt.title("Synthetic Dataset with Anomalies")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


In [ ]:
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Train an Isolation Forest model
isolation_forest = IsolationForest(random_state=42, contamination=0.05)
isolation_forest.fit(data_scaled)

# Predict anomalies
predictions = isolation_forest.predict(data_scaled)
predictions = np.where(predictions == 1, 0, 1)  # Convert to binary (0 = normal, 1 = anomaly)

# Visualize the predictions
plt.scatter(data[:, 0], data[:, 1], c=predictions, cmap='coolwarm', alpha=0.7)
plt.title("Anomalies Detected by Isolation Forest")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


In [ ]:
# Split data into training and testing
normal_data_train = normal_data[:800]
normal_data_test = normal_data[800:]
test_data = np.concatenate([normal_data_test, anomaly_data])

# Build the Autoencoder model
autoencoder = Sequential([
    Dense(16, activation='relu', input_dim=2),
    Dense(8, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='linear')  # Reconstruct the original input
])

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# Train the Autoencoder on normal data only
autoencoder.fit(normal_data_train, normal_data_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Compute reconstruction errors
reconstructions = autoencoder.predict(test_data)
errors = np.mean(np.square(test_data - reconstructions), axis=1)

# Set anomaly threshold
threshold = np.percentile(errors, 95)
print(f"Anomaly Threshold: {threshold}")

# Predict anomalies
autoencoder_predictions = errors > threshold

# Visualize the anomalies
plt.scatter(test_data[:, 0], test_data[:, 1], c=autoencoder_predictions, cmap='coolwarm', alpha=0.7)
plt.title("Anomalies Detected by Autoencoder")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
